In [ ]:
import os
import pandas as pd
from tqdm import tqdm
from functools import reduce
import numpy as np
import re

1、读取数据

In [ ]:
df1 = pd.read_excel("/Users/liyiman/Desktop/毕业论文/新的论文/复现备选/dataaaaa/供应链供应商关系.xlsx", dtype = {"统计截止日期": str, "股票代码": str, "公司股票代码": str, "供应商公司ID": str})
df2 = pd.read_excel("/Users/liyiman/Desktop/毕业论文/新的论文/复现备选/dataaaaa/供应链客户关系.xlsx", dtype = {"统计截止日期": str, "股票代码": str, "公司股票代码": str, "供应商公司ID": str})
df = pd.read_excel("/Users/liyiman/coding/NodeFormer/data/fraud/特征表.xlsx", dtype = {"证券代码": str, "会计年度": str} )
df["编号"] = df['会计年度'].astype(str) + df['证券代码'].astype(str)
columns = ['编号'] + [col for col in df.columns if col != '编号']
df = df[columns]

2、函数

In [ ]:
# 清洗关系表
def clean_df(df):
    df = df[df['统计截止日期'].str.endswith('12-31')]
    df['统计截止日期'] = pd.to_datetime(df['统计截止日期']).dt.year
    df = df[(df['统计截止日期'] >= 2003) & (df['统计截止日期'] <= 2023)]
    df = df[df['报表类型'] == 1]
    # 处理股票代码
    def process_code(code):
        code = str(code)  # 强制类型转换为 str
        if pd.isna(code):
            return code  # 跳过 NaN
        if ";" in code:
            return code.split(";")[0]
        return code
    df["公司股票代码"] = df["公司股票代码"].apply(process_code)
    # 替换规则：如果 "股票代码" 列不是空，则用 "股票代码" 的值替换 "编号" 列的对应行
    def not_empty(s):
        return bool(s.strip())
    df["供应商公司ID"] = df.apply(lambda row: row["公司股票代码"] if not_empty(row["公司股票代码"]) and row["公司股票代码"]!= "nan"  else row["供应商公司ID"], axis=1)
    # 补全供应商残缺
    df["编号"] = df["统计截止日期"].astype(str) + df["股票代码"].astype(str)
    return df

In [ ]:
# 生成平齐的5个公司
def creat_five(df):

    def process_company(group):
        num_rows = len(group)
        if num_rows == 1:
            # 插入 5 行，金额和比例平分
            amount = group.iloc[0]["供应商采购额"] / 5
            ratio = group.iloc[0]["供应商采购额占比"] / 5
            new_rows = [
                {**group.iloc[0].to_dict(), "排名": i + 1, "供应商采购额": amount, "供应商采购额占比": ratio}
                for i in range(5)
            ]
            return pd.concat([group, pd.DataFrame(new_rows)], ignore_index=True)
        elif 1 < num_rows < 6:
            # 插入 6 - i 行
            existing_amount = group[group["排名"] < 6]["供应商采购额"].sum()
            existing_ratio = group[group["排名"] < 6]["供应商采购额占比"].sum()
            # rank_6_rows = group[group["排名"] == 6]
            rank_1_rows = group[group["排名"] == 1]
            print("Rank 1 rows:", rank_1_rows)
            amount_to_distribute = group[group["排名"] == 6].iloc[0]["供应商采购额"] - existing_amount
            ratio_to_distribute = group[group["排名"] == 6].iloc[0]["供应商采购额占比"] - existing_ratio
            num_new_rows = 6 - num_rows
            amount = amount_to_distribute / num_new_rows
            ratio = ratio_to_distribute / num_new_rows
            new_rows = [
                {**group.iloc[0].to_dict(), "排名": num_rows + i, "供应商采购额": amount, "供应商采购额占比": ratio}
                for i in range(num_new_rows)
            ]
            return pd.concat([group, pd.DataFrame(new_rows)], ignore_index=True)
        else:
            return group[group["排名"] <= 6]
    # 按 "上市公司" 分组并生成平齐的5个公司
    df = df.groupby("编号", group_keys=False).apply(process_company)
    # 删除 "排名" 为 6 的行
    # df = df[df["排名"] < 6]
    return df


In [ ]:
# 将供应商和客户的交易情况匹配到特征表
# df1为特征表，df为关系表，entity_type为客户or供应商，返回df1特征表
def match_and_insert(df1, df, entity_type):
    for idx, row in df1.iterrows():
        matching_rows = df[df['编号'] == row['编号']]
        if not matching_rows.empty:
            # 按排名排序
            sorted_rows = matching_rows.sort_values(by='排名')
            for i in range(1, 6):
                rank_row = sorted_rows[sorted_rows['排名'] == i]
                if not rank_row.empty:
                    df1.at[idx, f'{entity_type}金额{i}'] = rank_row['供应商采购额'].values[0]
                    df1.at[idx, f'{entity_type}比例{i}'] = rank_row['供应商采购额占比'].values[0]
            # 插入总计
            total_row = sorted_rows[sorted_rows['排名'] == 6]
            if not total_row.empty:
                df1.at[idx, f'{entity_type}金额合计'] = total_row['供应商采购额'].values[0]
                df1.at[idx, f'{entity_type}比例合计'] = total_row['供应商采购额占比'].values[0]
    return df1

In [ ]:
# 清理和处理供应商名称杂乱问题
def clean_supplier_names(df):
    # 筛选个数大于6或上市公司的供应商名称
    df = df[(df['供应商名称'].str.len() > 6) | df['供应商公司ID'].notnull()]

    # 清理特殊符号和地名
    # df['供应商名称'] = df['供应商名称'].str.replace(r'[。，：:.,（【】）)([]{}*- —\\s]', '', regex=True)
    tqdm.pandas(desc="Cleaning symbols")
    df['供应商名称'] = df['供应商名称'].progress_apply(lambda x: re.sub(r'[。，：:.,（【】）)([]{}*- —\\s]', '', str(x)))
    # 清理
    province_names = ["北京市","天津市","上海市","重庆市","河北省","山西省","辽宁省","吉林省","黑龙江省","江苏省","浙江省","安徽省","福建省","江西省","山东省","河南省","湖北省","湖南省","广东省","海南省","四川省","贵州省","云南省","陕西省","甘肃省","青海省","台湾","内蒙古","广西","西藏","宁夏","新疆","香港","澳门", "北京","天津","上海","重庆","河北","山西","辽宁","吉林","黑龙江","江苏","浙江","安徽","福建","江西","山东","河南","湖北","湖南","广东","海南","四川","贵州","云南","陕西","甘肃","青海", "广州市","深圳市","成都市","杭州市","重庆市","武汉市","西安市","苏州市","天津市","南京市","郑州市","长沙市","东莞市","青岛市","佛山市","沈阳市","昆明市","宁波市","合肥市","无锡市","厦门市","哈尔滨市","济南市","福州市","南昌市","大连市","长春市","石家庄市","贵阳市","南宁市","太原市","乌鲁木齐市","常州市","温州市","珠海市","泉州市","金华市","惠州市","海口市","兰州市","徐州市","绍兴市","台州市","扬州市","烟台市","潍坊市","洛阳市","嘉兴市","泰州市","镇江市","南通市","威海市", "北京","上海","广州","深圳","成都","杭州","重庆","武汉","西安","苏州","天津","南京","郑州","长沙","东莞","青岛","佛山","沈阳","昆明","宁波","合肥","无锡","厦门","哈尔滨","济南","福州","南昌","大连","长春","石家庄","贵阳","南宁","太原","乌鲁木齐","常州","温州","珠海","泉州","金华","惠州","海口","兰州","徐州","绍兴","台州","扬州","烟台","潍坊","洛阳","嘉兴","泰州","镇江","南通","威海"]  # 示例地名，需补充完整
    province_pattern = '|'.join(province_names)  # 构造正则表达式
    # df['供应商名称'] = df['供应商名称'].str.replace(province_pattern, '', regex=True)
    tqdm.pandas(desc="Cleaning province names")
    df['供应商名称'] = df['供应商名称'].progress_apply(lambda x: re.sub(province_pattern, '', str(x)))

    # 前5个字相同的供应商归一化
    df['供应商名称'] = df['供应商名称'].str[:5]
    grouped = df.groupby('供应商名称')
    for name, group in tqdm(grouped, desc="Cleaning supplier names"):
        if group['供应商公司ID'].notnull().any():
            unified_name = group.loc[group['供应商公司ID'].notnull(), '供应商名称'].iloc[0]
        else:
            unified_name = group['供应商名称'].mode().iloc[0]
        df.loc[group.index, '供应商名称'] = unified_name
        df.loc[group.index, '供应商公司ID'] = group['供应商公司ID'].iloc[0] if group['供应商公司ID'].notnull().any() else np.nan

    # 为没有编号的供应商生成6位编号
    numeric_ids = df['供应商公司ID'].apply(lambda x: str(x).isdigit())
    max_existing_id = df.loc[numeric_ids, '供应商公司ID'].astype(int).max()
    # max_existing_id = df['供应商公司ID'].dropna().astype(int).max()
    new_id_start = max_existing_id + 1 if pd.notnull(max_existing_id) else 100000
    new_ids = {name: str(new_id_start + i) for i, name in enumerate(df['供应商名称'].unique()) if name not in df['供应商公司ID']}
    df['供应商公司ID'] = df.apply(lambda x: new_ids[x['供应商名称']] if pd.isnull(x['供应商公司ID']) else x['供应商公司ID'], axis=1)

    return df

In [ ]:
# 合并非上市公司到特征表
def sync_and_expand(df, df3):
    # 找到 df3 中 `供应编号` 不存在于 df 的 `编号` 中的行
    unmatched_rows = df3[~df3['供应商编号'].isin(df['编号'])]

    # 创建一个编号的分组
    grouped = unmatched_rows.groupby('供应商编号')

    for supply_id, group in grouped:
        if supply_id not in df['编号'].values:
            # 创建新行并添加编号
            new_row = {'编号': supply_id}
            
            # 动态加入供应商采购额和采购额占比列
            for idx, row in group.iterrows():
                new_row[f'非上市采购额{idx+1}'] = row['供应商采购额']
                new_row[f'非上市采购额占比{idx+1}'] = row['供应商采购额占比']
            
            # 将新行追加到 df 中
            df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    return df


3、调用函数

In [ ]:
df1 = creat_five(clean_df(df1))
df2 = creat_five(clean_df(df2))

In [ ]:
df = match_and_insert(df, df1, "供应商")
df = match_and_insert(df, df2, "客户")

In [ ]:
df3 = pd.concat([df1, df2], ignore_index=True)
df3 = clean_supplier_names(df3)
df3["供应商编号"] = df3['统计截止日期'].astype(str) + df3['供应商公司ID'].astype(str)

In [ ]:
df = sync_and_expand(df3, df)
columns_to_keep = ['编号', '供应商编号']
df3 = df3[columns_to_keep]

In [ ]:
df3.to_excel("/Users/liyiman/coding/NodeFormer/data/fraud/交易关系表.xlsx", index=False)
df.to_excel("/Users/liyiman/coding/NodeFormer/data/fraud/特征表+供应商关系+客户关系.xlsx", index=False)